# Megaline plan recomendator

## Contents

1. [Introduction](#introduction)
2. [Data Loading and Inspection]
3. [Model training]
    1. [Splitting the data into sets]
    2. [Tree model]
    3. [Forest model]
    4. [Logistic Regression model]
    5. [Quality check using the test set]
    6. [Sanity check]
4. [Conclusion]

## Introduction

This is the project for the "Intro into Machine Learning" sprint of Tripleten's DA course.

We will bw analizing user's data for the mobile carrier Megaline, in order to train a model that could properly recommend to each customer one of Megaline's new plans: Smart or Ultra.

The requested minimum accuracy for this model is **0.75**.

For this project we'll be using the following:
- Python 3.9.5
- Pandas 1.2.4
- Sklearn 0.24.1

Versions were chosen so they match as closely as possible the versions available on the Tripleten servers

In [3]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

[Back to Contents](#contents)